In [43]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from kneed import KneeLocator

import string
import re

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
lemmatizer = nltk.stem.WordNetLemmatizer()
nltk.download('averaged_perceptron_tagger')


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ekaterinamazur/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/ekaterinamazur/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [123]:
#read the pkl file for movies
df = pd.read_pickle('./ABC/full_movies.pkl')
#read the pkl file for shows
df_2 = pd.read_pickle('./ABC/full_shows.pkl')

In [124]:
#read json file activity for movies
activity_movie = pd.read_json('./movie_activities.json')
activity_shows = pd.read_json('./show_activities.json')

In [125]:
activity_movie.head()
activity_movie.groupby('user_id').count()

,content_id,activity,datetime
user_id,,,
1,213,213,213
2,199,199,199
3,190,190,190
4,181,181,181
5,217,217,217


## TF-IDF + kmeans clustering

In [126]:
stop_words = stopwords.words('english')
def clean_text(x):
    x = x.lower()
    x = x.translate(str.maketrans('', '', string.punctuation))
    # tokenize
    x = nltk.word_tokenize(x)
    x = [token for token in x if token not in stop_words]
    # lemmatization and pass string back
    x = ' '.join([lemmatizer.lemmatize(w) for w in x])
    return x

In [127]:
df['features'] = df['description'].apply(lambda x: clean_text(x))
df_2['features'] = df_2['description'].apply(lambda x: clean_text(x))

In [131]:
#tf-idf and kmeans for all movies
# Convert the text data to a matrix of TF-IDF features
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(df['features'])


inertia = []
#calculate the optimal number of clusters
K = range(1, 80)
for k in K:
    kmeanModel = KMeans(n_clusters=k,random_state=0)
    kmeanModel.fit(tfidf)
    inertia.append(kmeanModel.inertia_)


kn = KneeLocator(K, inertia, curve='convex', direction='decreasing')
print(kn.knee)


None


/Users/ekaterinamazur/anaconda3/lib/python3.9/site-packages/kneed/knee_locator.py:311: UserWarning: No knee/elbow found
  warnings.warn("No knee/elbow found")


In [132]:
#tf-idf and kmeans for all tv-shows
# Convert the text data to a matrix of TF-IDF features
tfidf_vectorizer_shows = TfidfVectorizer(stop_words='english')
tfidf_shows = tfidf_vectorizer_shows.fit_transform(df_2['features'])


inertia = []
#calculate the optimal number of clusters
K = range(1, 20)
for k in K:
    kmeanModel = KMeans(n_clusters=k,random_state=0)
    kmeanModel.fit(tfidf_shows)
    inertia.append(kmeanModel.inertia_)


kn = KneeLocator(K, inertia, curve='convex', direction='decreasing')
print(kn.knee)

7


In [154]:
# Cluster the documents using KMeans
num_clusters = 33
km = KMeans(n_clusters=num_clusters)
km.fit(tfidf)

# Assign cluster labels to each document
clusters = km.labels_.tolist()
df['cluster'] = clusters
df.head()

,id,category,title,series,episode_name,description,description2,tags,image,directors_actors,tags2,publication_date,rating,duration_sec,season,episode,episode_title,features,cluster
0,0,Documentary,The Making Of David Attenborough's Conquest Of...,NaN,The Making Of David Attenborough's Conquest Of...,Witness the incredible innovations and technic...,Witness the incredible innovations and technic...,"[ABC TV Plus, DOCUMENTARY]",https://cdn.iview.abc.net.au/thumbs/i/zx/ZX970...,[[]],"[abc2, docs, nature, uk, featured, david-atten...",2021-03-01 07:00:00,G,3159.0,NaN,,The Making Of David Attenborough's Conquest Of...,witness incredible innovation technical advanc...,24
1,1,Documentary,Ballet Now,NaN,Ballet Now,This film captures the creative process behind...,This film captures the creative process behind...,"[ABC TV Plus, DOCUMENTARY, ARTS & CULTURE, FAM...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW269...,[[]],"[abc2, docs, factual, arts, ballet, dance, cul...",2021-06-30 21:59:27,M,3555.0,NaN,,Ballet Now,film capture creative process behind scene fam...,19
2,2,Documentary,Freddie Mercury: The Final Act,NaN,Freddie Mercury: The Final Act,The extraordinary story of Freddie Mercury's b...,The extraordinary story of Freddie Mercury's b...,"[ABC TV, DOCUMENTARY, ARTS & CULTURE]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW324...,[[]],"[abc1, docs, arts, factual, culture, music, ro...",2022-03-15 20:31:03,M,5377.0,NaN,,Freddie Mercury: The Final Act,extraordinary story freddie mercury battle aid...,18
3,3,Documentary,Fly Me To The Moon,NaN,Fly Me To The Moon,The Moon Landing was the iconic moment of our ...,The Moon Landing was the iconic moment of our ...,"[ABC NEWS, DOCUMENTARY, SCIENCE]",https://cdn.iview.abc.net.au/thumbs/i/nc/NC193...,[[]],"[news24, news, space, docs, science, featured]",2021-04-07 13:00:00,PG,3480.0,NaN,,Fly Me To The Moon,moon landing iconic moment time john barron ex...,18
4,4,Documentary,Weird Australia,NaN,Weird Australia,"With the help of eyewitness accounts, qualifie...","With the help of eyewitness accounts, qualifie...","[ABC TV, SCIENCE, REGIONAL AUSTRALIA, DOCUMENT...",https://cdn.iview.abc.net.au/thumbs/i/sq/SQ210...,[['Host Moataz Hamde']],"[abc1, science, australia, nature, wildlife, e...",2022-01-01 09:29:00,PG,1802.0,NaN,,Weird Australia,help eyewitness account qualified expert amate...,24


In [134]:
# Cluster the documents using KMeans
num_clusters_shows = 7
km_shows = KMeans(n_clusters=num_clusters_shows)
km_shows.fit(tfidf_shows)

# Assign cluster labels to each document
clusters_shows = km_shows.labels_.tolist()
df_2['cluster'] = clusters_shows
df_2.reset_index(inplace=True)
df_2.head()

,index,id,title,category,description,description2,directors_actors,publication_date,rating,duration_sec,tags,tags2,image,features,cluster
0,0,10000,199 Little Heroes,Education,The journey to school is a very special type o...,"[Enjo lives in Quinten, a forest glade in the ...",[[]],2021-06-01 06:20:00,G,"[311.0, 311.0, 312.0, 311.0, 311.0, 320.0, 311...","[ABC ME, EDUCATION]","[abc3, primary-humanities, environment-day, re...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW073...,journey school special type life experience sh...,2
1,1,10001,7.30,News,Leigh Sales presents Australia's leading night...,[The devastation caused by the recent floods i...,[[]],2022-03-14 20:00:00,None,"[1906.0, 1894.0, 1879.0, 1935.0, 1891.0, 1822....","[ABC TV, ABC NEWS]","[budget, economy, 730, abc-faves, covid-19, ne...",https://cdn.iview.abc.net.au/thumbs/i/nc/NC220...,leigh sale present australia leading nightly p...,3
2,2,10002,7.30 Mark Humphries Satire,News,Satirist Mark Humphries brings his unique pers...,[Satirist Mark Humphries goes inside the Healt...,[[]],2021-05-11 14:00:00,None,"[145.0, 110.0, 118.0, 136.0, 202.0, 115.0, 241...",[ABC NEWS],"[mark, ley, satire, humphries, care, duty, of,...",https://cdn.iview.abc.net.au/thumbs/i/nn/NN211...,satirist mark humphries brings unique perspect...,3
3,3,10003,7.30: The Interviews,News,Revisit some of 7.30's most compelling convers...,[Musician Dave Grohl sits down with 7.30's Lei...,[[]],2022-03-08 20:00:00,None,[1030.0],[ABC NEWS],"[news24, news]",https://cdn.iview.abc.net.au/thumbs/i/ns/NS224...,revisit 730s compelling conversation australia...,4
4,4,10004,72 Cutest Animals,Family,The animal kingdom continually astounds us wit...,[Cute is a notion that can mean many things to...,[[]],2022-03-05 15:03:06,G,"[1616.0, 1616.0, 1620.0, 1620.0, 1614.0, 1620....","[ABC ME, FAMILY]","[abc3, family-viewing-abcme, family-viewing]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW243...,animal kingdom continually astounds u showcase...,5


In [155]:
australian = 'Australia|australia|australian|Australian|Aboriginal|Indigenous|First People'

In [156]:
expat_library = df[df['tags2'].str.contains(australian) | df['description'].str.contains(australian) | df['description2'].str.contains(australian) | df['title'].str.contains(australian)
]
expat_library.head()

,id,category,title,series,episode_name,description,description2,tags,image,directors_actors,tags2,publication_date,rating,duration_sec,season,episode,episode_title,features,cluster
3,3,Documentary,Fly Me To The Moon,NaN,Fly Me To The Moon,The Moon Landing was the iconic moment of our ...,The Moon Landing was the iconic moment of our ...,"[ABC NEWS, DOCUMENTARY, SCIENCE]",https://cdn.iview.abc.net.au/thumbs/i/nc/NC193...,[[]],"[news24, news, space, docs, science, featured]",2021-04-07 13:00:00,PG,3480.0,NaN,,Fly Me To The Moon,moon landing iconic moment time john barron ex...,18
4,4,Documentary,Weird Australia,NaN,Weird Australia,"With the help of eyewitness accounts, qualifie...","With the help of eyewitness accounts, qualifie...","[ABC TV, SCIENCE, REGIONAL AUSTRALIA, DOCUMENT...",https://cdn.iview.abc.net.au/thumbs/i/sq/SQ210...,[['Host Moataz Hamde']],"[abc1, science, australia, nature, wildlife, e...",2022-01-01 09:29:00,PG,1802.0,NaN,,Weird Australia,help eyewitness account qualified expert amate...,24
8,8,Documentary,Ready Together,NaN,Ready Together,Craig Reucassel presents the stories of two co...,Craig Reucassel presents the stories of two co...,"[ABC TV, DOCUMENTARY, LIFESTYLE, SCIENCE]",https://cdn.iview.abc.net.au/thumbs/i/do/DO202...,[['Host Craig Reucassel']],"[abc1, docs, lifestyle, factual, nature, weath...",2021-03-28 18:28:08,PG,1500.0,NaN,,Ready Together,craig reucassel present story two community de...,5
13,13,Documentary,Cyber Dreaming,NaN,Cyber Dreaming,Graphic artist Stuart Campbell has developed h...,Graphic artist Stuart Campbell has developed h...,"[ABC TV, ABC ARTS, ARTS & CULTURE, DOCUMENTARY]",https://cdn.iview.abc.net.au/thumbs/i/ac/publi...,[[]],"[abc1, abcarts, arts, docs, aussie, naidoc, hi...",2018-07-11 14:00:50,G,1570.0,NaN,,Cyber Dreaming,graphic artist stuart campbell developed uniqu...,18
14,14,Documentary,I'm Wanita,NaN,I'm Wanita,"The story of Australia's Queen of Honky Tonk, ...","The story of Australia's Queen of Honky Tonk, ...","[ABC TV Plus, ABC TV, DOCUMENTARY, ARTS & CULT...",https://cdn.iview.abc.net.au/thumbs/i/ac/AC182...,[[]],"[abc2, abc1, docs, factual, abcarts, arts, cul...",2022-04-01 20:32:00,M,3525.0,NaN,,I'm Wanita,story australia queen honky tonk wanita bahtiy...,30


In [157]:
expat_library_shows = df_2[df_2['tags2'].str.contains(australian) | df_2['description'].str.contains(australian) | df_2['title'].str.contains(australian)
]
expat_library_shows

,level_0,id,title,category,description,description2,directors_actors,publication_date,rating,duration_sec,tags,tags2,image,features,cluster
56,56,10056,Australia Debates,Panel Discussion,The funniest minds in Australia debate the fin...,"[Tom Cashman, Demi Lardner and Nat Damena argu...",[['Host Nikki Britton']],2021-06-28 21:33:30,M,"[2579.0, 2605.0, 2492.0]","[ABC TV Plus, PANEL & DISCUSSION]","[ns:be-informed, panel, public-affairs, aussie...",https://cdn.iview.abc.net.au/thumbs/i/fr/FR201...,funniest mind australia debate finding three i...,6
57,57,10057,Australia Remastered: Australia's Oceans,Family,Explore the incredible diversity of ocean wild...,[The Southern Ocean is the only ocean that str...,[['Host Aaron Pedersen']],2020-10-25 19:03:00,G,"[3230.0, 3195.0, 3185.0]","[ABC TV, ABC ME, DOCUMENTARY, SCIENCE, FAMILY]","[environment, family-viewing-abcme, docs, ocea...",https://cdn.iview.abc.net.au/thumbs/i/do/DO184...,explore incredible diversity ocean wildlife en...,0
58,58,10058,Australia Remastered: Forces of Nature,Documentary,"Hosted and narrated by Aaron Pedersen, this se...",[Cyclones burst upon Australia's tropical coas...,[['Host Aaron Pedersen']],2022-01-30 18:04:10,G,"[3230.0, 3260.0, 3195.0, 3240.0]","[ABC TV, DOCUMENTARY, REGIONAL AUSTRALIA, FAMILY]","[environment, animals, informative, family-vie...",https://cdn.iview.abc.net.au/thumbs/i/do/DO184...,hosted narrated aaron pedersen series explores...,1
59,59,10059,Australia Remastered: Nature's Great Divide,Documentary,The hard line that separates the natural world...,[The narrow strait between the Indonesian isla...,[['Host Aaron Pedersen']],2021-12-07 21:27:05,G,"[3240.0, 3240.0, 3200.0]","[ABC TV, DOCUMENTARY, SCIENCE, FAMILY]","[environment, animals, informative, family-vie...",https://cdn.iview.abc.net.au/thumbs/i/do/DO195...,hard line separate natural world asia australi...,0
60,60,10060,Australia Remastered: Wild Australians,Family,Explore some of Australia's most iconic and fa...,"[Echidnas and platypuses are unique, the only ...",[['Host Aaron Pedersen']],2020-08-30 18:57:00,G,"[3310.0, 3330.0, 3285.0, 3185.0, 3230.0, 3294.0]","[ABC TV, ABC ME, DOCUMENTARY, FAMILY]","[environment, family-viewing-abcme, science-we...",https://cdn.iview.abc.net.au/thumbs/i/do/DO184...,explore australia iconic fascinating animal my...,4
61,61,10061,Australia Remastered: Wild Treasures,Family,Australia's diverse and unique landscapes and ...,[The Daintree Rainforest has one of the highes...,[['Host Aaron Pedersen']],2021-01-11 21:22:47,G,"[3205.0, 3315.0, 3260.0, 3254.0, 3257.0, 3220.0]","[ABC TV, ABC ME, DOCUMENTARY, FAMILY]","[environment, family-viewing-abcme, science-we...",https://cdn.iview.abc.net.au/thumbs/i/do/DO184...,australia diverse unique landscape ecosystem u...,0
62,62,10062,Australia Remastered: Wild Wars,Documentary,Life in Australia is a battle. Shaped by the c...,[While forests only cover 16% of Australia's l...,[['Host Aaron Pedersen']],2021-03-02 22:33:27,G,"[3235.0, 3265.0, 3280.0, 3240.0]","[ABC TV, DOCUMENTARY, FAMILY]","[environment, science-week, docs, ocean, famil...",https://cdn.iview.abc.net.au/thumbs/i/do/DO184...,life australia battle shaped challenge epic en...,5
63,63,10063,Australia's Ocean Odyssey: A Journey Down The ...,Documentary,A landmark documentary series that takes a spe...,[Journey down the East Australian Current all ...,[['Narrator Marta Dusseldorp']],2020-06-09 21:33:25,G,"[3549.0, 360.0, 3515.0, 3446.0]","[ABC TV, ABC ME, DOCUMENTARY, SCIENCE, FAMILY]","[abc3, environment, animals, informative, fami...",https://cdn.iview.abc.net.au/thumbs/i/do/DO180...,landmark documentary series take spectacular j...,0
64,64,10064,Australia's Prime Ministers,Education,"This series profiles Australia's leaders, feat...","[This series profiles Australia's leaders, fea...",[[]],2022-02-28 09:00:00,G,"[183.0, 119.0, 186.0, 180.0, 199.0, 227.0, 157...","[ABC ME, EDUCATION, DOCUMENTARY]","[abc3, primary-humanities, gough-whitlam, educ...",https

In [158]:
#reset index for df_2
df_2.reset_index(inplace=True)
#drop the column 'index'
df_2.drop(columns=['index'], inplace=True)
df_2.head()

,level_0,id,title,category,description,description2,directors_actors,publication_date,rating,duration_sec,tags,tags2,image,features,cluster
0,0,10000,199 Little Heroes,Education,The journey to school is a very special type o...,"[Enjo lives in Quinten, a forest glade in the ...",[[]],2021-06-01 06:20:00,G,"[311.0, 311.0, 312.0, 311.0, 311.0, 320.0, 311...","[ABC ME, EDUCATION]","[abc3, primary-humanities, environment-day, re...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW073...,journey school special type life experience sh...,2
1,1,10001,7.30,News,Leigh Sales presents Australia's leading night...,[The devastation caused by the recent floods i...,[[]],2022-03-14 20:00:00,None,"[1906.0, 1894.0, 1879.0, 1935.0, 1891.0, 1822....","[ABC TV, ABC NEWS]","[budget, economy, 730, abc-faves, covid-19, ne...",https://cdn.iview.abc.net.au/thumbs/i/nc/NC220...,leigh sale present australia leading nightly p...,3
2,2,10002,7.30 Mark Humphries Satire,News,Satirist Mark Humphries brings his unique pers...,[Satirist Mark Humphries goes inside the Healt...,[[]],2021-05-11 14:00:00,None,"[145.0, 110.0, 118.0, 136.0, 202.0, 115.0, 241...",[ABC NEWS],"[mark, ley, satire, humphries, care, duty, of,...",https://cdn.iview.abc.net.au/thumbs/i/nn/NN211...,satirist mark humphries brings unique perspect...,3
3,3,10003,7.30: The Interviews,News,Revisit some of 7.30's most compelling convers...,[Musician Dave Grohl sits down with 7.30's Lei...,[[]],2022-03-08 20:00:00,None,[1030.0],[ABC NEWS],"[news24, news]",https://cdn.iview.abc.net.au/thumbs/i/ns/NS224...,revisit 730s compelling conversation australia...,4
4,4,10004,72 Cutest Animals,Family,The animal kingdom continually astounds us wit...,[Cute is a notion that can mean many things to...,[[]],2022-03-05 15:03:06,G,"[1616.0, 1616.0, 1620.0, 1620.0, 1614.0, 1620....","[ABC ME, FAMILY]","[abc3, family-viewing-abcme, family-viewing]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW243...,animal kingdom continually astounds u showcase...,5


In [159]:
expat_library_shows= df_2[df_2['tags2'].str.contains(australian) | df_2['description'].str.contains(australian) | df_2['description2'].str.contains(australian) | df_2['title'].str.contains(australian)
]
expat_library_shows.head()

,level_0,id,title,category,description,description2,directors_actors,publication_date,rating,duration_sec,tags,tags2,image,features,cluster
56,56,10056,Australia Debates,Panel Discussion,The funniest minds in Australia debate the fin...,"[Tom Cashman, Demi Lardner and Nat Damena argu...",[['Host Nikki Britton']],2021-06-28 21:33:30,M,"[2579.0, 2605.0, 2492.0]","[ABC TV Plus, PANEL & DISCUSSION]","[ns:be-informed, panel, public-affairs, aussie...",https://cdn.iview.abc.net.au/thumbs/i/fr/FR201...,funniest mind australia debate finding three i...,6
57,57,10057,Australia Remastered: Australia's Oceans,Family,Explore the incredible diversity of ocean wild...,[The Southern Ocean is the only ocean that str...,[['Host Aaron Pedersen']],2020-10-25 19:03:00,G,"[3230.0, 3195.0, 3185.0]","[ABC TV, ABC ME, DOCUMENTARY, SCIENCE, FAMILY]","[environment, family-viewing-abcme, docs, ocea...",https://cdn.iview.abc.net.au/thumbs/i/do/DO184...,explore incredible diversity ocean wildlife en...,0
58,58,10058,Australia Remastered: Forces of Nature,Documentary,"Hosted and narrated by Aaron Pedersen, this se...",[Cyclones burst upon Australia's tropical coas...,[['Host Aaron Pedersen']],2022-01-30 18:04:10,G,"[3230.0, 3260.0, 3195.0, 3240.0]","[ABC TV, DOCUMENTARY, REGIONAL AUSTRALIA, FAMILY]","[environment, animals, informative, family-vie...",https://cdn.iview.abc.net.au/thumbs/i/do/DO184...,hosted narrated aaron pedersen series explores...,1
59,59,10059,Australia Remastered: Nature's Great Divide,Documentary,The hard line that separates the natural world...,[The narrow strait between the Indonesian isla...,[['Host Aaron Pedersen']],2021-12-07 21:27:05,G,"[3240.0, 3240.0, 3200.0]","[ABC TV, DOCUMENTARY, SCIENCE, FAMILY]","[environment, animals, informative, family-vie...",https://cdn.iview.abc.net.au/thumbs/i/do/DO195...,hard line separate natural world asia australi...,0
60,60,10060,Australia Remastered: Wild Australians,Family,Explore some of Australia's most iconic and fa...,"[Echidnas and platypuses are unique, the only ...",[['Host Aaron Pedersen']],2020-08-30 18:57:00,G,"[3310.0, 3330.0, 3285.0, 3185.0, 3230.0, 3294.0]","[ABC TV, ABC ME, DOCUMENTARY, FAMILY]","[environment, family-viewing-abcme, science-we...",https://cdn.iview.abc.net.au/thumbs/i/do/DO184...,explore australia iconic fascinating animal my...,4


In [160]:
# Print the top 10 terms for each cluster
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
terms = tfidf_vectorizer.get_feature_names()
for i in range(num_clusters):
    print("Cluster %d:" % i, end='')
    for ind in order_centroids[i, :13]:
        print(' %s' % terms[ind], end='')
    print()



Cluster 0: knew father identity travel fielding home buckley young jaydyn australia understand try man
Cluster 1: family love new funny stick story life thomas 2010 2008 home moving shoe
Cluster 2: life forever strange 2006 year group jane changed hang men spend terror project
Cluster 3: scientist world greatest 2009 epic master quest international deep secret story known 2011
Cluster 4: lead left splat club long jack football truth beloved professionally gifted 1979 personally
Cluster 5: film summer body try pdhpe story gender period stage transition english community zog
Cluster 6: art 2004 dragged adviser whelan murray detective political assassination assignment hitman longing beguiling
Cluster 7: geography 45 science eruption stage program impact desertification cycle carbon city asks volcanic
Cluster 8: mother child baby chimpanzee raised experiment home born language happened julian fin philomena
Cluster 9: beach deserted neville gave cadet recall kangaroo flashback stranded los

/Users/ekaterinamazur/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [161]:
# Print the top 10 terms for each cluster
order_centroids_shows = km_shows.cluster_centers_.argsort()[:, ::-1]
terms_shows = tfidf_vectorizer_shows.get_feature_names()
for i in range(num_clusters_shows):
    print("Cluster %d:" % i, end='')
    for ind in order_centroids_shows[i, :13]:
        print(' %s' % terms_shows[ind], end='')
    print()

Cluster 0: life world new australia friend team play join story best return family art
Cluster 1: stage explores work world future series english history extraordinary life 45 new australian
Cluster 2: adventure child book life little stage based series funny love story school follow
Cluster 3: news abc issue brings story reporting today original exclusive presented investigation matter political
Cluster 4: australian science teacher skill teach lesson australia family tenminute experiment history fundamental journey
Cluster 5: series young live fun world follows life people adventure everyday story friend stage
Cluster 6: big kid question word answer australia australian year wacky world asking animal podcast


In [162]:
joined_activity = pd.merge(df, activity_movie, left_on='id', right_on='content_id', how='right')
#make from list in tag2 column to string and delete [] in the string
joined_activity['tags2'] = joined_activity['tags2'].apply(lambda x: str(x).replace('[', '').replace(']', ''))
#drop NaN values in id column
joined_activity.dropna(subset=['id'], inplace=True)
#drop columns that are not needed
joined_activity.drop(['series', 'episode_name', 'description2', 'publication_date', 'rating', 'season', 'episode', 'episode_title', 'datetime', 'duration_sec', 'tags'], axis=1, inplace=True)
joined_activity.head()

,id,category,title,description,image,directors_actors,tags2,features,cluster,content_id,activity,user_id
24,295.0,Kids,See Us Coming Together: A Sesame Street Special,"It's ""Neighbour Day"" on Sesame Street and ever...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW325...,[[]],'abc4kids',neighbour day sesame street everyone something...,10.0,295,Like,4
43,95.0,Documentary,Girl Like You,"Over six years, a couple battles to stay toget...",https://cdn.iview.abc.net.au/thumbs/i/do/DO203...,[[]],"'abc2', 'docs', 'factual', 'identity', 'transg...",six year couple battle stay together one trans...,5.0,95,Dislike,1
53,278.0,Family,Here Comes The Grump,An evil wizard presides over the entire world ...,https://cdn.iview.abc.net.au/thumbs/i/zw/ZW275...,[[]],"'abc3', '3featured', 'family-viewing-abcme', '...",evil wizard presides entire world banished hap...,17.0,278,Dislike,5
55,107.0,Drama,Adaptation,A lovelorn screenwriter struggles to adapt 'Th...,https://cdn.iview.abc.net.au/thumbs/i/zw/ZW300...,[['Director Spike Jonze']],"'abc1', 'abc2', 'drama', 'comedy-drama', 'surr...",lovelorn screenwriter struggle adapt orchid th...,2.0,107,Like,2
61,207.0,Drama,Redfern Now (2014),When a young Aboriginal woman is assaulted and...,https://cdn.iview.abc.net.au/thumbs/i/ip/IP130...,"[['Cast Deb Mailman', 'Wayne Blair', 'Rarriwuy...","'abc1', 'aussie', 'drama', 'a:established-pare...",young aboriginal woman assaulted doesnt report...,32.0,207,Like,1


In [163]:
joined_activity_shows = pd.merge(df_2, activity_shows, left_on='id', right_on='content_id', how='right')
#make from list in tag2 column to string and delete [] in the string
joined_activity_shows['tags2'] = joined_activity_shows['tags2'].apply(lambda x: str(x).replace('[', '').replace(']', ''))
#drop NaN values in id column
joined_activity_shows.dropna(subset=['id'], inplace=True)
#drop columns that are not needed
joined_activity_shows.drop([ 'description2', 'publication_date', 'rating', 'datetime', 'duration_sec', 'tags'], axis=1, inplace=True)
joined_activity_shows.head()

,level_0,id,title,category,description,directors_actors,tags2,image,features,cluster,content_id,activity,user_id
0,649.0,10649.0,What It's Like,Education,A show for young people in our community whose...,[[]],"'abc3', 'me-prod', 'family-viewing-abcme', 'ch...",https://cdn.iview.abc.net.au/thumbs/i/ck/CK200...,show young people community whose voice story ...,5.0,10649,Play,1
1,536.0,10536.0,The Crown and Us: The Story of The Royals in A...,Documentary,The Crown is part of who we were and who we ar...,[['Director Bernice Toni']],"'history', 'compelling', 'informative', 'biogr...",https://cdn.iview.abc.net.au/thumbs/i/do/DO171...,crown part despite secret scandal question rel...,6.0,10536,Dislike,1
2,425.0,10425.0,Play School Nursery Rhyme News Time,Kids,"Ever wondered why the cow jumps over the moon,...",[[]],"'abc4kids', '4learn', '4play-school', 'childre...",https://cdn.iview.abc.net.au/thumbs/i/ck/CK190...,ever wondered cow jump moon little bo peep she...,2.0,10425,Like,1
3,594.0,10594.0,The Wacky Word Show,Education,"The Wacky Word Show features skits, songs, ani...",[[]],"'abc3', 'education', 'primary-english'",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW263...,wacky word show feature skit song animation sp...,6.0,10594,Like,2
5,219.0,10219.0,Giggle And Hoot,Kids,Jimmy Giggle and all his owl pals have awesome...,[[]],"'childrens', 'abc4kids', '4best', '4lower'",https://cdn.iview.abc.net.au/thumbs/i/ck/CK183...,jimmy giggle owl pal awesome adventure fun tog...,5.0,10219,Play,2


In [164]:
#select rows where activity is 'Like'
expat_data_likes = joined_activity[joined_activity['activity'] == 'Like']
expat_data_likes.sort_values(by=['user_id'])

,id,category,title,description,image,directors_actors,tags2,features,cluster,content_id,activity,user_id
598,37.0,Documentary,Catching A Predator,The inside story of the investigation into ser...,https://cdn.iview.abc.net.au/thumbs/i/zw/ZW321...,[[]],"'abc1', 'uk', 'docs', 'true-crime', 'factual',...",inside story investigation serial male rapist ...,26.0,37,Like,1
61,207.0,Drama,Redfern Now (2014),When a young Aboriginal woman is assaulted and...,https://cdn.iview.abc.net.au/thumbs/i/ip/IP130...,"[['Cast Deb Mailman', 'Wayne Blair', 'Rarriwuy...","'abc1', 'aussie', 'drama', 'a:established-pare...",young aboriginal woman assaulted doesnt report...,32.0,207,Like,1
80,110.0,Drama,A Raisin In The Sun,When a recently widowed matriarch receives a l...,https://cdn.iview.abc.net.au/thumbs/i/zw/ZW301...,[['Director Daniel Petrie']],"'abc1', 'abc2', 'drama', 'film', 'feature-leng...",recently widowed matriarch receives large sum ...,25.0,110,Like,1
947,35.0,Documentary,Brazen Hussies,A revealing documentary celebrating the legacy...,https://cdn.iview.abc.net.au/thumbs/i/do/DO190...,[['Director Catherine Dwyer']],"'abc1', 'featured', 'docs', 'australia', 'fact...",revealing documentary celebrating legacy bold ...,32.0,35,Like,1
692,280.0,Family,Manou The Swift,The little orphaned swift Manou is adopted by ...,https://cdn.iview.abc.net.au/thumbs/i/zw/ZW299...,[[]],"'abc3', 'family-viewing'",little orphaned swift manou adopted seagull co...,26.0,280,Like,1
859,155.0,Drama,My Girl,A tender coming-of-age story that follows Vada...,https://cdn.iview.abc.net.au/thumbs/i/zw/ZW301...,[['Director Howard Zieff']],"'abc1', 'abc2', 'drama', 'comedy-drama', '1990...",tender comingofage story follows vada 11yearol...,5.0,155,Like,1
735,187.0,Drama,Jack Irish: Dead Point,When a high profile judge commissions Jack Iri...,https://cdn.iview.abc.net.au/thumbs/i/dr/DR122...,"[['Cast Guy Pearce', 'Marta Dusseldorp', 'Aaro...","'iview', 'drama', 'crime', 'courts', 'aussie',...",high profile judge commission jack irish locat...,4.0,187,Like,1
227,118.0,Drama,The Last Station,A historical drama that illustrates Russian au...,https://cdn.iview.abc.net.au/thumbs/i/zw/ZW278...,[['Director Michael Hoffman']],"'abc1', 'abc2', 'film', 'drama', 'romance', 'a...",historical drama illustrates russian author le...,3.0,118,Like,1
701,7.0,Documentary,Armadillo: Narrated By David Attenborough,In the heart of Brazil lives an animal so elus...,https://cdn.iview.abc.net.au/thumbs/i/zw/ZW311...,[['Host Sir David Attenborough']],"'abc1', 'uk', 'docs', 'factual', 'nature', 'wi...",heart brazil life animal elusive ever seen wil...,27.0,7,Like,1
731,135.0,Drama,Trust The Man,"Two New York couples; a successful actress, he...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW286...,[['Director Bart Freundlich']],"'abc1', 'abc2', 'comedy', 'drama', 'film', 'ro...",two new york couple successful actress househu...,31.0,135,Like,1


In [165]:
#select rows where activity is 'Like'
expat_data_shows_likes = joined_activity_shows[joined_activity_shows['activity'] == 'Like']
expat_data_shows_likes.sort_values(by=['user_id'])

,level_0,id,title,category,description,directors_actors,tags2,image,features,cluster,content_id,activity,user_id
2,425.0,10425.0,Play School Nursery Rhyme News Time,Kids,"Ever wondered why the cow jumps over the moon,...",[[]],"'abc4kids', '4learn', '4play-school', 'childre...",https://cdn.iview.abc.net.au/thumbs/i/ck/CK190...,ever wondered cow jump moon little bo peep she...,2.0,10425,Like,1
405,65.0,10065.0,Australian Of The Year Profiles 2022,Arts,Celebrate the achievements of our 2022 Austral...,[[]],"'sustainability', 'environment', 'queensland',...",https://cdn.iview.abc.net.au/thumbs/i/rk/RK210...,celebrate achievement 2022 australian year you...,4.0,10065,Like,1
1063,3.0,10003.0,7.30: The Interviews,News,Revisit some of 7.30's most compelling convers...,[[]],"'news24', 'news'",https://cdn.iview.abc.net.au/thumbs/i/ns/NS224...,revisit 730s compelling conversation australia...,4.0,10003,Like,1
1068,80.0,10080.0,Barrenjoey Road,Documentary,This three part series follows the true crime ...,"[['Host Ruby Jones', 'Neil Mercer']]","'true-crime', 'a:older-sin', 'a:established-pa...",https://cdn.iview.abc.net.au/thumbs/i/do/DO171...,three part series follows true crime story bar...,5.0,10080,Like,1
368,538.0,10538.0,The Crystal Maze,Family,"A team of five contestants, led by an eccentri...",[['Host Richard Ayoade']],"'abc3', 'abc3-faves', 'family-viewing-abcme', ...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW195...,team five contestant led eccentric maze master...,0.0,10538,Like,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
520,473.0,10473.0,Same But Different,Education,Short documentary portraits of primary childre...,[[]],"'abc3', 'mental-as', 'education', '\nc:3childr...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW008...,short documentary portrait primary child range...,2.0,10473,Like,5
848,507.0,10507.0,Spirit Riding Free,Family,A wild mustang Spirit meets gutsy 12-year old ...,[[]],"'abc3', 'family-viewing', '3featured', 'childr...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW115...,wild mustang spirit meet gutsy 12year old luck...,5.0,10507,Like,5
1172,209.0,10209.0,Further Back In Time For Dinner (Simplified Ch...,Family,An ordinary family travels back in time 120 ye...,[[]],"'abc1', 'docs', 'family-viewing'",https://cdn.iview.abc.net.au/thumbs/i/iv/IV201...,ordinary family travel back time 120 year imme...,4.0,10209,Like,5
1160,476.0,10476.0,Sarah And Duck,Kids,Sarah is a 7 year old girl with big eyes and a...,[[]],"'abc4kids', 'childrens', '4gentle', '4best-fri...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW122...,sarah 7 year old girl big eye green hat life q...,6.0,10476,Like,5


In [166]:
#top-3 clusters for each user in expat_data_likes
top3 = expat_data_likes.groupby('user_id')['cluster'].value_counts().groupby(level=0).nlargest(3).reset_index(level=1, drop=True)
#make from series to dataframe
top3 = top3.to_frame()
#drop the 3rd column
top3 = top3.drop(columns=['cluster'])
#make user_id column as index
top3 = top3.reset_index()
top3

,user_id,cluster
0,1,26.0
1,1,32.0
2,1,3.0
3,2,25.0
4,2,3.0
5,2,2.0
6,3,3.0
7,3,18.0
8,3,20.0
9,4,2.0


In [167]:
expat_data_shows_likes.sort_values(by=['user_id'])
#top-3 clusters for each user in expat_data_likes
top3_show = expat_data_shows_likes.groupby('user_id')['cluster'].value_counts().groupby(level=0).nlargest(3).reset_index(level=1,
                                                                                                              drop=True)
#make from series to dataframe
top3_show = top3_show.to_frame()
#drop the 3rd column
top3_show = top3_show.drop(columns=['cluster'])
#make user_id column as index
top3_show = top3_show.reset_index()
top3_show

,user_id,cluster
0,1,0.0
1,1,5.0
2,1,4.0
3,2,0.0
4,2,5.0
5,2,4.0
6,3,0.0
7,3,5.0
8,3,4.0
9,4,0.0


In [178]:
#join top 3 clusters for each user with expat_library
recommended= pd.merge(top3, expat_library, on=['cluster'], how='left')
recommended

,user_id,cluster,id,category,title,series,episode_name,description,description2,tags,image,directors_actors,tags2,publication_date,rating,duration_sec,season,episode,episode_title,features
0,1,26.0,32.0,Documentary,Capturing Cricket: Steve Waugh In India,NaN,Capturing Cricket: Steve Waugh In India,"Steve Waugh, the mastermind behind Australia's...","Steve Waugh, the mastermind behind Australia's...","[ABC TV, DOCUMENTARY, SPORT]",https://cdn.iview.abc.net.au/thumbs/i/rf/RF192...,[['Host Steve Waugh']],"[abc1, australia, docs, factual, sport, divers...",2020-11-17 21:30:47,PG,3407.0,NaN,,Capturing Cricket: Steve Waugh In India,steve waugh mastermind behind australia golden...
1,1,26.0,80.0,Documentary,Women Of Steel,NaN,Women Of Steel,Outraged at sexual harassment in the work plac...,Outraged at sexual harassment in the work plac...,"[ABC TV Plus, DOCUMENTARY]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW292...,[[]],"[abc2, australia, docs, factual, history, inte...",2022-03-06 20:32:41,M,3325.0,NaN,,Women Of Steel,outraged sexual harassment work place denied w...
2,1,32.0,35.0,Documentary,Brazen Hussies,NaN,Brazen Hussies,A revealing documentary celebrating the legacy...,A revealing documentary celebrating the legacy...,"[ABC TV, DOCUMENTARY]",https://cdn.iview.abc.net.au/thumbs/i/do/DO190...,[['Director Catherine Dwyer']],"[abc1, featured, docs, australia, factual, his...",2022-03-08 01:04:06,M,5318.0,NaN,,Brazen Hussies,revealing documentary celebrating legacy bold ...
3,1,32.0,49.0,Documentary,Bronwyn Oliver: The Shadows Within,NaN,Bronwyn Oliver: The Shadows Within,This intimate documentary follows the journey ...,This intimate documentary follows the journey ...,"[ABC TV Plus, ABC ARTS, ARTS & CULTURE, DOCUME...",https://cdn.iview.abc.net.au/thumbs/i/ac/AC201...,[[]],"[abc2, arts, abcarts, docs, factual, new-south...",2021-10-13 21:03:54,PG,3567.0,NaN,,Bronwyn Oliver: The Shadows Within,intimate documentary follows journey bronwyn o...
4,1,32.0,158.0,Drama,Sisters Of War,NaN,Sisters Of War,Inspired by the remarkable true story of two e...,Inspired by the remarkable true story of two e...,"[ABC TV, DRAMA]",https://cdn.iview.abc.net.au/thumbs/i/dr/DR090...,[['Director Brendan Maher']],"[abc1, drama, history, a:established-parents, ...",2021-06-11 07:00:00,M,5725.0,NaN,,Sisters Of War,inspired remarkable true story two extraordina...
5,1,32.0,207.0,Drama,Redfern Now (2014),NaN,Redfern Now (2014),When a young Aboriginal woman is assaulted and...,When a young Aboriginal woman is assaulted and...,"[ABC TV, DRAMA, INDIGENOUS]",https://cdn.iview.abc.net.au/thumbs/i/ip/IP130...,"[['Cast Deb Mailman', 'Wayne Blair', 'Rarriwuy...","[abc1, aussie, drama, a:established-parents, b...",2021-06-30 07:00:00,M,5194.0,NaN,,Redfern Now (),young aboriginal woman assaulted doesnt report...
6,1,3.0,23.0,Documentary,Can We Save The Reef?,NaN,Can We Save The Reef?,An epic story of Australian and international ...,An epic story of Australian and international ...,"[ABC TV, DOCUMENTARY, SCIENCE, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/sc/SC160...,[[]],"[abc1, docs, nature, science, science-week, cl...",2018-05-12 14:58:55,G,3428.0,NaN,,Can We Save The Reef?,epic story australian international scientist ...
7,1,3.0,48.0,Documentary,Cracking COVID,NaN,Cracking COVID,Tracking the real-time story of Australia's sc...,Tracking the real-time story of Australia's sc...,"[ABC TV, DOCUMENTARY, SCIENCE]",https://cdn.iview.abc.net.au/thumbs/i/do/DO200...,[['Director Sonya Pemberton']],"[abc1, aussie, factual, docs, science, health,...",2021-07-13 20:29:59,PG,4496.0,NaN,,Cracking COVID,tracking realtime story australia scientific r...
8,1,3.0,265.0,Family,Australia: The Wild Top End,NaN,Australia: The Wild Top End,Come with us on an epic journey through some o...,Come with us on an epic journey through some o...,"[ABC ME, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW284...,[[]],"[abc3, 3featured, family-viewing-abcme, family...",2022-03-16 09:45:0

In [169]:
#shows
recommended_shows= pd.merge(top3_show, expat_library_shows, on=['cluster'], how='left')
recommended_shows

,user_id,cluster,level_0,id,title,category,description,description2,directors_actors,publication_date,rating,duration_sec,tags,tags2,image,features
0,1,0.0,57,10057,Australia Remastered: Australia's Oceans,Family,Explore the incredible diversity of ocean wild...,[The Southern Ocean is the only ocean that str...,[['Host Aaron Pedersen']],2020-10-25 19:03:00,G,"[3230.0, 3195.0, 3185.0]","[ABC TV, ABC ME, DOCUMENTARY, SCIENCE, FAMILY]","[environment, family-viewing-abcme, docs, ocea...",https://cdn.iview.abc.net.au/thumbs/i/do/DO184...,explore incredible diversity ocean wildlife en...
1,1,0.0,59,10059,Australia Remastered: Nature's Great Divide,Documentary,The hard line that separates the natural world...,[The narrow strait between the Indonesian isla...,[['Host Aaron Pedersen']],2021-12-07 21:27:05,G,"[3240.0, 3240.0, 3200.0]","[ABC TV, DOCUMENTARY, SCIENCE, FAMILY]","[environment, animals, informative, family-vie...",https://cdn.iview.abc.net.au/thumbs/i/do/DO195...,hard line separate natural world asia australi...
2,1,0.0,61,10061,Australia Remastered: Wild Treasures,Family,Australia's diverse and unique landscapes and ...,[The Daintree Rainforest has one of the highes...,[['Host Aaron Pedersen']],2021-01-11 21:22:47,G,"[3205.0, 3315.0, 3260.0, 3254.0, 3257.0, 3220.0]","[ABC TV, ABC ME, DOCUMENTARY, FAMILY]","[environment, family-viewing-abcme, science-we...",https://cdn.iview.abc.net.au/thumbs/i/do/DO184...,australia diverse unique landscape ecosystem u...
3,1,0.0,63,10063,Australia's Ocean Odyssey: A Journey Down The ...,Documentary,A landmark documentary series that takes a spe...,[Journey down the East Australian Current all ...,[['Narrator Marta Dusseldorp']],2020-06-09 21:33:25,G,"[3549.0, 360.0, 3515.0, 3446.0]","[ABC TV, ABC ME, DOCUMENTARY, SCIENCE, FAMILY]","[abc3, environment, animals, informative, fami...",https://cdn.iview.abc.net.au/thumbs/i/do/DO180...,landmark documentary series take spectacular j...
4,1,0.0,212,10212,Gardening Australia - My Garden Path,Arts,Gardening Australia is hosted by Costa Georgia...,[We meet a garden designer who celebrates text...,[[]],2021-02-12 07:00:00,G,"[366.0, 372.0, 407.0, 509.0, 385.0, 467.0, 401...","[ABC TV, LIFESTYLE, REGIONAL AUSTRALIA]","[home, environment, queensland, western-austra...",https://cdn.iview.abc.net.au/thumbs/i/rk/RK190...,gardening australia hosted costa georgiadis sc...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,5,4.0,176,10176,Employable Me Australia,Documentary,The Rose d'Or nominated series follows six ind...,"[This uplifting, warm, and insightful series s...",[[]],2019-04-09 21:32:30,PG,"[239.0, 168.0, 3566.0, 3579.0, 235.0, 233.0, 3...","[ABC TV, DOCUMENTARY]","[heartfelt, ns:be-uplifted, informative, docus...",https://cdn.iview.abc.net.au/thumbs/i/dc/DC182...,rose dor nominated series follows six individu...
66,5,4.0,354,10354,Miriam Margolyes Almost Australian,Documentary,A revealing documentary series; following cele...,[Miriam is leaving the home she shares with he...,[['Host Miriam Margolyes']],2021-06-18 07:00:00,M,"[3539.0, 3442.0, 3431.0]","[ABC TV, DOCUMENTARY]","[adventure, charming, real-life, outback, ns:b...",https://cdn.iview.abc.net.au/thumbs/i/do/DO190...,revealing documentary series following celebra...
67,5,4.0,549,10549,The Great Australian Bee Challenge,Documentary,"In this two-part special, host Paul West chall...","[In the first of two programs, host Paul West ...",[['Host Paul West']],2019-01-29 21:33:06,G,"[3594.0, 3336.0]","[ABC TV, DOCUMENTARY, SCIENCE, FAMILY]","[technology, ns:be-entertained, compelling, fa...",https://cdn.iview.abc.net.au/thumbs/i/sq/SQ180...,twopart special host paul west challenge four ...
68,5,4.0,652,10652,When War Came To Australia,News,Australians are aware of the Bombing of Darwin...,[In 1944 hundreds of captured Japanese prisone...,[[]],2019-08-05 11:00:00,None,"[292.0, 141.0, 118.0]","[ABC TV, ABC NEWS]","[abc1, history, news]",https://cdn.iview.abc.net.au/thumbs/i/nn/N

In [170]:
#drop intersections of recommended and activity by user_id and content_id
recommended = recommended[~recommended[['user_id', 'id']].apply(frozenset, axis=1).isin(activity[['user_id', 'content_id']].apply(frozenset, axis=1))]
recommended

,user_id,cluster,id,category,title,series,episode_name,description,description2,tags,image,directors_actors,tags2,publication_date,rating,duration_sec,season,episode,episode_title,features
0,1,26.0,32.0,Documentary,Capturing Cricket: Steve Waugh In India,NaN,Capturing Cricket: Steve Waugh In India,"Steve Waugh, the mastermind behind Australia's...","Steve Waugh, the mastermind behind Australia's...","[ABC TV, DOCUMENTARY, SPORT]",https://cdn.iview.abc.net.au/thumbs/i/rf/RF192...,[['Host Steve Waugh']],"[abc1, australia, docs, factual, sport, divers...",2020-11-17 21:30:47,PG,3407.0,NaN,,Capturing Cricket: Steve Waugh In India,steve waugh mastermind behind australia golden...
1,1,26.0,80.0,Documentary,Women Of Steel,NaN,Women Of Steel,Outraged at sexual harassment in the work plac...,Outraged at sexual harassment in the work plac...,"[ABC TV Plus, DOCUMENTARY]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW292...,[[]],"[abc2, australia, docs, factual, history, inte...",2022-03-06 20:32:41,M,3325.0,NaN,,Women Of Steel,outraged sexual harassment work place denied w...
2,1,32.0,35.0,Documentary,Brazen Hussies,NaN,Brazen Hussies,A revealing documentary celebrating the legacy...,A revealing documentary celebrating the legacy...,"[ABC TV, DOCUMENTARY]",https://cdn.iview.abc.net.au/thumbs/i/do/DO190...,[['Director Catherine Dwyer']],"[abc1, featured, docs, australia, factual, his...",2022-03-08 01:04:06,M,5318.0,NaN,,Brazen Hussies,revealing documentary celebrating legacy bold ...
3,1,32.0,49.0,Documentary,Bronwyn Oliver: The Shadows Within,NaN,Bronwyn Oliver: The Shadows Within,This intimate documentary follows the journey ...,This intimate documentary follows the journey ...,"[ABC TV Plus, ABC ARTS, ARTS & CULTURE, DOCUME...",https://cdn.iview.abc.net.au/thumbs/i/ac/AC201...,[[]],"[abc2, arts, abcarts, docs, factual, new-south...",2021-10-13 21:03:54,PG,3567.0,NaN,,Bronwyn Oliver: The Shadows Within,intimate documentary follows journey bronwyn o...
4,1,32.0,158.0,Drama,Sisters Of War,NaN,Sisters Of War,Inspired by the remarkable true story of two e...,Inspired by the remarkable true story of two e...,"[ABC TV, DRAMA]",https://cdn.iview.abc.net.au/thumbs/i/dr/DR090...,[['Director Brendan Maher']],"[abc1, drama, history, a:established-parents, ...",2021-06-11 07:00:00,M,5725.0,NaN,,Sisters Of War,inspired remarkable true story two extraordina...
5,1,32.0,207.0,Drama,Redfern Now (2014),NaN,Redfern Now (2014),When a young Aboriginal woman is assaulted and...,When a young Aboriginal woman is assaulted and...,"[ABC TV, DRAMA, INDIGENOUS]",https://cdn.iview.abc.net.au/thumbs/i/ip/IP130...,"[['Cast Deb Mailman', 'Wayne Blair', 'Rarriwuy...","[abc1, aussie, drama, a:established-parents, b...",2021-06-30 07:00:00,M,5194.0,NaN,,Redfern Now (),young aboriginal woman assaulted doesnt report...
6,1,3.0,23.0,Documentary,Can We Save The Reef?,NaN,Can We Save The Reef?,An epic story of Australian and international ...,An epic story of Australian and international ...,"[ABC TV, DOCUMENTARY, SCIENCE, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/sc/SC160...,[[]],"[abc1, docs, nature, science, science-week, cl...",2018-05-12 14:58:55,G,3428.0,NaN,,Can We Save The Reef?,epic story australian international scientist ...
7,1,3.0,48.0,Documentary,Cracking COVID,NaN,Cracking COVID,Tracking the real-time story of Australia's sc...,Tracking the real-time story of Australia's sc...,"[ABC TV, DOCUMENTARY, SCIENCE]",https://cdn.iview.abc.net.au/thumbs/i/do/DO200...,[['Director Sonya Pemberton']],"[abc1, aussie, factual, docs, science, health,...",2021-07-13 20:29:59,PG,4496.0,NaN,,Cracking COVID,tracking realtime story australia scientific r...
8,1,3.0,265.0,Family,Australia: The Wild Top End,NaN,Australia: The Wild Top End,Come with us on an epic journey through some o...,Come with us on an epic journey through some o...,"[ABC ME, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW284...,[[]],"[abc3, 3featured, family-viewing-abcme, family...",2022-03-16 09:45:0

In [171]:
recommended_shows = recommended_shows[~recommended_shows[['user_id', 'id']].apply(frozenset, axis=1).isin(activity[['user_id', 'content_id']].apply(frozenset, axis=1))]
recommended_shows

,user_id,cluster,level_0,id,title,category,description,description2,directors_actors,publication_date,rating,duration_sec,tags,tags2,image,features
0,1,0.0,57,10057,Australia Remastered: Australia's Oceans,Family,Explore the incredible diversity of ocean wild...,[The Southern Ocean is the only ocean that str...,[['Host Aaron Pedersen']],2020-10-25 19:03:00,G,"[3230.0, 3195.0, 3185.0]","[ABC TV, ABC ME, DOCUMENTARY, SCIENCE, FAMILY]","[environment, family-viewing-abcme, docs, ocea...",https://cdn.iview.abc.net.au/thumbs/i/do/DO184...,explore incredible diversity ocean wildlife en...
1,1,0.0,59,10059,Australia Remastered: Nature's Great Divide,Documentary,The hard line that separates the natural world...,[The narrow strait between the Indonesian isla...,[['Host Aaron Pedersen']],2021-12-07 21:27:05,G,"[3240.0, 3240.0, 3200.0]","[ABC TV, DOCUMENTARY, SCIENCE, FAMILY]","[environment, animals, informative, family-vie...",https://cdn.iview.abc.net.au/thumbs/i/do/DO195...,hard line separate natural world asia australi...
2,1,0.0,61,10061,Australia Remastered: Wild Treasures,Family,Australia's diverse and unique landscapes and ...,[The Daintree Rainforest has one of the highes...,[['Host Aaron Pedersen']],2021-01-11 21:22:47,G,"[3205.0, 3315.0, 3260.0, 3254.0, 3257.0, 3220.0]","[ABC TV, ABC ME, DOCUMENTARY, FAMILY]","[environment, family-viewing-abcme, science-we...",https://cdn.iview.abc.net.au/thumbs/i/do/DO184...,australia diverse unique landscape ecosystem u...
3,1,0.0,63,10063,Australia's Ocean Odyssey: A Journey Down The ...,Documentary,A landmark documentary series that takes a spe...,[Journey down the East Australian Current all ...,[['Narrator Marta Dusseldorp']],2020-06-09 21:33:25,G,"[3549.0, 360.0, 3515.0, 3446.0]","[ABC TV, ABC ME, DOCUMENTARY, SCIENCE, FAMILY]","[abc3, environment, animals, informative, fami...",https://cdn.iview.abc.net.au/thumbs/i/do/DO180...,landmark documentary series take spectacular j...
4,1,0.0,212,10212,Gardening Australia - My Garden Path,Arts,Gardening Australia is hosted by Costa Georgia...,[We meet a garden designer who celebrates text...,[[]],2021-02-12 07:00:00,G,"[366.0, 372.0, 407.0, 509.0, 385.0, 467.0, 401...","[ABC TV, LIFESTYLE, REGIONAL AUSTRALIA]","[home, environment, queensland, western-austra...",https://cdn.iview.abc.net.au/thumbs/i/rk/RK190...,gardening australia hosted costa georgiadis sc...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,5,4.0,176,10176,Employable Me Australia,Documentary,The Rose d'Or nominated series follows six ind...,"[This uplifting, warm, and insightful series s...",[[]],2019-04-09 21:32:30,PG,"[239.0, 168.0, 3566.0, 3579.0, 235.0, 233.0, 3...","[ABC TV, DOCUMENTARY]","[heartfelt, ns:be-uplifted, informative, docus...",https://cdn.iview.abc.net.au/thumbs/i/dc/DC182...,rose dor nominated series follows six individu...
66,5,4.0,354,10354,Miriam Margolyes Almost Australian,Documentary,A revealing documentary series; following cele...,[Miriam is leaving the home she shares with he...,[['Host Miriam Margolyes']],2021-06-18 07:00:00,M,"[3539.0, 3442.0, 3431.0]","[ABC TV, DOCUMENTARY]","[adventure, charming, real-life, outback, ns:b...",https://cdn.iview.abc.net.au/thumbs/i/do/DO190...,revealing documentary series following celebra...
67,5,4.0,549,10549,The Great Australian Bee Challenge,Documentary,"In this two-part special, host Paul West chall...","[In the first of two programs, host Paul West ...",[['Host Paul West']],2019-01-29 21:33:06,G,"[3594.0, 3336.0]","[ABC TV, DOCUMENTARY, SCIENCE, FAMILY]","[technology, ns:be-entertained, compelling, fa...",https://cdn.iview.abc.net.au/thumbs/i/sq/SQ180...,twopart special host paul west challenge four ...
68,5,4.0,652,10652,When War Came To Australia,News,Australians are aware of the Bombing of Darwin...,[In 1944 hundreds of captured Japanese prisone...,[[]],2019-08-05 11:00:00,None,"[292.0, 141.0, 118.0]","[ABC TV, ABC NEWS]","[abc1, history, news]",https://cdn.iview.abc.net.au/thumbs/i/nn/N

In [179]:
#choose 2 rows per each user_id and each cluster
recommended_films = recommended.groupby(['user_id', 'cluster']).head(2)
recommended_films

,user_id,cluster,id,category,title,series,episode_name,description,description2,tags,image,directors_actors,tags2,publication_date,rating,duration_sec,season,episode,episode_title,features
0,1,26.0,32.0,Documentary,Capturing Cricket: Steve Waugh In India,NaN,Capturing Cricket: Steve Waugh In India,"Steve Waugh, the mastermind behind Australia's...","Steve Waugh, the mastermind behind Australia's...","[ABC TV, DOCUMENTARY, SPORT]",https://cdn.iview.abc.net.au/thumbs/i/rf/RF192...,[['Host Steve Waugh']],"[abc1, australia, docs, factual, sport, divers...",2020-11-17 21:30:47,PG,3407.0,NaN,,Capturing Cricket: Steve Waugh In India,steve waugh mastermind behind australia golden...
1,1,26.0,80.0,Documentary,Women Of Steel,NaN,Women Of Steel,Outraged at sexual harassment in the work plac...,Outraged at sexual harassment in the work plac...,"[ABC TV Plus, DOCUMENTARY]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW292...,[[]],"[abc2, australia, docs, factual, history, inte...",2022-03-06 20:32:41,M,3325.0,NaN,,Women Of Steel,outraged sexual harassment work place denied w...
2,1,32.0,35.0,Documentary,Brazen Hussies,NaN,Brazen Hussies,A revealing documentary celebrating the legacy...,A revealing documentary celebrating the legacy...,"[ABC TV, DOCUMENTARY]",https://cdn.iview.abc.net.au/thumbs/i/do/DO190...,[['Director Catherine Dwyer']],"[abc1, featured, docs, australia, factual, his...",2022-03-08 01:04:06,M,5318.0,NaN,,Brazen Hussies,revealing documentary celebrating legacy bold ...
3,1,32.0,49.0,Documentary,Bronwyn Oliver: The Shadows Within,NaN,Bronwyn Oliver: The Shadows Within,This intimate documentary follows the journey ...,This intimate documentary follows the journey ...,"[ABC TV Plus, ABC ARTS, ARTS & CULTURE, DOCUME...",https://cdn.iview.abc.net.au/thumbs/i/ac/AC201...,[[]],"[abc2, arts, abcarts, docs, factual, new-south...",2021-10-13 21:03:54,PG,3567.0,NaN,,Bronwyn Oliver: The Shadows Within,intimate documentary follows journey bronwyn o...
6,1,3.0,23.0,Documentary,Can We Save The Reef?,NaN,Can We Save The Reef?,An epic story of Australian and international ...,An epic story of Australian and international ...,"[ABC TV, DOCUMENTARY, SCIENCE, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/sc/SC160...,[[]],"[abc1, docs, nature, science, science-week, cl...",2018-05-12 14:58:55,G,3428.0,NaN,,Can We Save The Reef?,epic story australian international scientist ...
7,1,3.0,48.0,Documentary,Cracking COVID,NaN,Cracking COVID,Tracking the real-time story of Australia's sc...,Tracking the real-time story of Australia's sc...,"[ABC TV, DOCUMENTARY, SCIENCE]",https://cdn.iview.abc.net.au/thumbs/i/do/DO200...,[['Director Sonya Pemberton']],"[abc1, aussie, factual, docs, science, health,...",2021-07-13 20:29:59,PG,4496.0,NaN,,Cracking COVID,tracking realtime story australia scientific r...
9,2,25.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,2,3.0,23.0,Documentary,Can We Save The Reef?,NaN,Can We Save The Reef?,An epic story of Australian and international ...,An epic story of Australian and international ...,"[ABC TV, DOCUMENTARY, SCIENCE, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/sc/SC160...,[[]],"[abc1, docs, nature, science, science-week, cl...",2018-05-12 14:58:55,G,3428.0,NaN,,Can We Save The Reef?,epic story australian international scientist ...
11,2,3.0,48.0,Documentary,Cracking COVID,NaN,Cracking COVID,Tracking the real-time story of Australia's sc...,Tracking the real-time story of Australia's sc...,"[ABC TV, DOCUMENTARY, SCIENCE]",https://cdn.iview.abc.net.au/thumbs/i/do/DO200...,[['Director Sonya Pemberton']],"[abc1, aussie, factual, docs, science, health,...",2021-07-13 20:29:59,PG,4496.0,NaN,,Cracking COVID,tracking realtime story australia scientific r...
13,2,2.0,34.0,Documentary,New Dads,NaN,New Dads,Jane Caro meets five Australian men navigating...,Jane Caro meets five Australian men navigating...,"[ABC TV, DOCUMENTARY]",https://cdn.iview.abc.net.au/thumbs/i/rn/RN181...,[['Host Jane Caro

In [180]:
recommended_shows = recommended_shows.groupby(['user_id', 'cluster']).head(2)
recommended_shows

,user_id,cluster,level_0,id,title,category,description,description2,directors_actors,publication_date,rating,duration_sec,tags,tags2,image,features
0,1,0.0,57,10057,Australia Remastered: Australia's Oceans,Family,Explore the incredible diversity of ocean wild...,[The Southern Ocean is the only ocean that str...,[['Host Aaron Pedersen']],2020-10-25 19:03:00,G,"[3230.0, 3195.0, 3185.0]","[ABC TV, ABC ME, DOCUMENTARY, SCIENCE, FAMILY]","[environment, family-viewing-abcme, docs, ocea...",https://cdn.iview.abc.net.au/thumbs/i/do/DO184...,explore incredible diversity ocean wildlife en...
1,1,0.0,59,10059,Australia Remastered: Nature's Great Divide,Documentary,The hard line that separates the natural world...,[The narrow strait between the Indonesian isla...,[['Host Aaron Pedersen']],2021-12-07 21:27:05,G,"[3240.0, 3240.0, 3200.0]","[ABC TV, DOCUMENTARY, SCIENCE, FAMILY]","[environment, animals, informative, family-vie...",https://cdn.iview.abc.net.au/thumbs/i/do/DO195...,hard line separate natural world asia australi...
6,1,5.0,62,10062,Australia Remastered: Wild Wars,Documentary,Life in Australia is a battle. Shaped by the c...,[While forests only cover 16% of Australia's l...,[['Host Aaron Pedersen']],2021-03-02 22:33:27,G,"[3235.0, 3265.0, 3280.0, 3240.0]","[ABC TV, DOCUMENTARY, FAMILY]","[environment, science-week, docs, ocean, famil...",https://cdn.iview.abc.net.au/thumbs/i/do/DO184...,life australia battle shaped challenge epic en...
7,1,4.0,60,10060,Australia Remastered: Wild Australians,Family,Explore some of Australia's most iconic and fa...,"[Echidnas and platypuses are unique, the only ...",[['Host Aaron Pedersen']],2020-08-30 18:57:00,G,"[3310.0, 3330.0, 3285.0, 3185.0, 3230.0, 3294.0]","[ABC TV, ABC ME, DOCUMENTARY, FAMILY]","[environment, family-viewing-abcme, science-we...",https://cdn.iview.abc.net.au/thumbs/i/do/DO184...,explore australia iconic fascinating animal my...
8,1,4.0,65,10065,Australian Of The Year Profiles 2022,Arts,Celebrate the achievements of our 2022 Austral...,"[A materials scientist, engineer and inventor ...",[[]],2022-01-05 17:00:00,G,"[197.0, 209.0, 210.0, 198.0, 181.0, 196.0, 210...","[ABC TV, DOCUMENTARY]","[sustainability, environment, queensland, west...",https://cdn.iview.abc.net.au/thumbs/i/rk/RK210...,celebrate achievement 2022 australian year you...
13,2,0.0,57,10057,Australia Remastered: Australia's Oceans,Family,Explore the incredible diversity of ocean wild...,[The Southern Ocean is the only ocean that str...,[['Host Aaron Pedersen']],2020-10-25 19:03:00,G,"[3230.0, 3195.0, 3185.0]","[ABC TV, ABC ME, DOCUMENTARY, SCIENCE, FAMILY]","[environment, family-viewing-abcme, docs, ocea...",https://cdn.iview.abc.net.au/thumbs/i/do/DO184...,explore incredible diversity ocean wildlife en...
14,2,0.0,59,10059,Australia Remastered: Nature's Great Divide,Documentary,The hard line that separates the natural world...,[The narrow strait between the Indonesian isla...,[['Host Aaron Pedersen']],2021-12-07 21:27:05,G,"[3240.0, 3240.0, 3200.0]","[ABC TV, DOCUMENTARY, SCIENCE, FAMILY]","[environment, animals, informative, family-vie...",https://cdn.iview.abc.net.au/thumbs/i/do/DO195...,hard line separate natural world asia australi...
19,2,5.0,62,10062,Australia Remastered: Wild Wars,Documentary,Life in Australia is a battle. Shaped by the c...,[While forests only cover 16% of Australia's l...,[['Host Aaron Pedersen']],2021-03-02 22:33:27,G,"[3235.0, 3265.0, 3280.0, 3240.0]","[ABC TV, DOCUMENTARY, FAMILY]","[environment, science-week, docs, ocean, famil...",https://cdn.iview.abc.net.au/thumbs/i/do/DO184...,life australia battle shaped challenge epic en...
20,2,4.0,60,10060,Australia Remastered: Wild Australians,Family,Explore some of Australia's most iconic and fa...,"[Echidnas and platypuses are unique, the only ...",[['Host Aaron Pedersen']],2020-08-30 18:57:00,G,"[3310.0, 3330.0, 3285.0, 3185.0, 3230.0, 3294.0]","[ABC TV, ABC ME, DOCUMENTARY, FAMILY]","[environment, family-viewing-abcme, science-we...",

In [187]:
#drop na in id column and mix recommended_films and recommended_shows
recommended_films.dropna(subset=['id'], inplace=True)
recommended_shows.dropna(subset=['id'], inplace=True)
#join recommended_films and recommended_shows
recommended_mix = pd.concat([recommended_films, recommended_shows])
#choose the random 6 rows per each user_id
recommended_mix = recommended_mix.groupby('user_id').apply(lambda x: x.sample(6, replace=False))
recommended_mix.reset_index(drop=True, inplace=True)


/var/folders/x6/923dwt55237392mlk00827nh0000gn/T/ipykernel_76033/1059194490.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recommended_films.dropna(subset=['id'], inplace=True)


In [188]:
recommended_mix

,user_id,cluster,id,category,title,series,episode_name,description,description2,tags,...,directors_actors,tags2,publication_date,rating,duration_sec,season,episode,episode_title,features,level_0
0,1,26.0,80.0,Documentary,Women Of Steel,NaN,Women Of Steel,Outraged at sexual harassment in the work plac...,Outraged at sexual harassment in the work plac...,"[ABC TV Plus, DOCUMENTARY]",...,[[]],"[abc2, australia, docs, factual, history, inte...",2022-03-06 20:32:41,M,3325.0,NaN,,Women Of Steel,outraged sexual harassment work place denied w...,NaN
1,1,32.0,35.0,Documentary,Brazen Hussies,NaN,Brazen Hussies,A revealing documentary celebrating the legacy...,A revealing documentary celebrating the legacy...,"[ABC TV, DOCUMENTARY]",...,[['Director Catherine Dwyer']],"[abc1, featured, docs, australia, factual, his...",2022-03-08 01:04:06,M,5318.0,NaN,,Brazen Hussies,revealing documentary celebrating legacy bold ...,NaN
2,1,4.0,10065.0,Arts,Australian Of The Year Profiles 2022,NaN,NaN,Celebrate the achievements of our 2022 Austral...,"[A materials scientist, engineer and inventor ...","[ABC TV, DOCUMENTARY]",...,[[]],"[sustainability, environment, queensland, west...",2022-01-05 17:00:00,G,"[197.0, 209.0, 210.0, 198.0, 181.0, 196.0, 210...",NaN,NaN,NaN,celebrate achievement 2022 australian year you...,65.0
3,1,4.0,10060.0,Family,Australia Remastered: Wild Australians,NaN,NaN,Explore some of Australia's most iconic and fa...,"[Echidnas and platypuses are unique, the only ...","[ABC TV, ABC ME, DOCUMENTARY, FAMILY]",...,[['Host Aaron Pedersen']],"[environment, family-viewing-abcme, science-we...",2020-08-30 18:57:00,G,"[3310.0, 3330.0, 3285.0, 3185.0, 3230.0, 3294.0]",NaN,NaN,NaN,explore australia iconic fascinating animal my...,60.0
4,1,32.0,49.0,Documentary,Bronwyn Oliver: The Shadows Within,NaN,Bronwyn Oliver: The Shadows Within,This intimate documentary follows the journey ...,This intimate documentary follows the journey ...,"[ABC TV Plus, ABC ARTS, ARTS & CULTURE, DOCUME...",...,[[]],"[abc2, arts, abcarts, docs, factual, new-south...",2021-10-13 21:03:54,PG,3567.0,NaN,,Bronwyn Oliver: The Shadows Within,intimate documentary follows journey bronwyn o...,NaN
5,1,3.0,23.0,Documentary,Can We Save The Reef?,NaN,Can We Save The Reef?,An epic story of Australian and international ...,An epic story of Australian and international ...,"[ABC TV, DOCUMENTARY, SCIENCE, FAMILY]",...,[[]],"[abc1, docs, nature, science, science-week, cl...",2018-05-12 14:58:55,G,3428.0,NaN,,Can We Save The Reef?,epic story australian international scientist ...,NaN
6,2,0.0,10057.0,Family,Australia Remastered: Australia's Oceans,NaN,NaN,Explore the incredible diversity of ocean wild...,[The Southern Ocean is the only ocean that str...,"[ABC TV, ABC ME, DOCUMENTARY, SCIENCE, FAMILY]",...,[['Host Aaron Pedersen']],"[environment, family-viewing-abcme, docs, ocea...",2020-10-25 19:03:00,G,"[3230.0, 3195.0, 3185.0]",NaN,NaN,NaN,explore incredible diversity ocean wildlife en...,57.0
7,2,5.0,10062.0,Documentary,Australia Remastered: Wild Wars,NaN,NaN,Life in Australia is a battle. Shaped by the c...,[While forests only cover 16% of Australia's l...,"[ABC TV, DOCUMENTARY, FAMILY]",...,[['Host Aaron Pedersen']],"[environment, science-week, docs, ocean, famil...",2021-03-02 22:33:27,G,"[3235.0, 3265.0, 3280.0, 3240.0]",NaN,NaN,NaN,life australia battle shaped challenge epic en...,62.0
8,2,4.0,10065.0,Arts,Australian Of The Year Profiles 2022,NaN,NaN,Celebrate the achievements of our 2022 Austral...,"[A materials scientist, engineer and inventor ...","[ABC TV, DOCUMENTARY]",...,[[]],"[sustainability, environment, queensland, west...",2022-01-05 17:00:00,G,"[197.0, 209.0, 210.0, 198.0, 181.0, 196.0, 210...",NaN,NaN,NaN,celebrate achievement 2022 australian year you...,65.0
9,2,3.0,23.0,Documentary,Can We Save The Reef?,NaN,Can We Save The Reef?,An epic story of Australian and international ...,An epic story of Australian and international ...,"[ABC TV, DOCUMENTARY, SCIENCE, FAMI